In [1]:
import redis
import json
import requests
from bs4 import BeautifulSoup
import httpx
import time
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.nn import functional as F
from collections import defaultdict
from string import punctuation
from nltk.corpus import stopwords
from boilerpy3 import extractors
import spacy
from spacy.lang.en import English
from spacy.lang.ru import Russian
from nltk.corpus import wordnet as wn
from textblob import TextBlob
import tldextract
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import nltk
from urllib.parse import urlparse
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy
import matplotlib.pyplot as plt
import datetime
import joblib
from collections import Counter

path = '/home/kalacheva/Desktop/'

import sys

import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

from sklearn.metrics import classification_report

In [ ]:
# Here we have dataset from dump:
df = pd.read_csv(path + 'ten_million_url_list.csv')

# Split it for train and test:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# And save it in different files (they contain too much values):
train_df.to_csv(path + 'train.csv', index=False)
test_df.to_csv(path + 'test.csv', index=False)

In [11]:
# But we want to have only 28 categories (check excel file for information):

category_mapping = {
    'Information Technology': 'Internet and Telecom',
    'Search Engines and Portals': 'Internet and Telecom',
    'Internet Telephony': 'Internet and Telecom',
    'Freeware and Software Downloads': 'Internet and Telecom',
    'Entertainment': 'Arts and Entertainment',
    'Arts and Culture': 'Arts and Entertainment',
    'Internet Radio and TV': 'Arts and Entertainment',
    'Sports Hunting and War Games': 'Arts and Entertainment',
    'Newsgroups and Message Boards': 'Arts and Entertainment',
    'Business': 'Business and Industry',
    'Travel': 'Travel',
    'Health': 'Health',
    'Health and Wellness': 'Health',
    'Medicine': 'Health',
    'Games': 'Games',
    'Society and Lifestyles': 'People and Society',
    'Personal Websites and Blogs': 'People and Society',
    'Social Networking': 'People and Society',
    'Finance and Banking': 'Finance',
    'Brokerage and Trading': 'Finance',
    'Auction': 'Finance',
    'Sports': 'Sports',
    'News and Media': 'News and Media',
    'Education': 'Career and Education',
    'Child Education': 'Career and Education',
    'Gambling': 'Gambling',
    'Restaurant and Dining': 'Food and Drink',
    'Personal Vehicles': 'Autos and Vehicles',
    'Government and Legal Organizations': 'Law and Government',
    'Political Organizations': 'Law and Government',
    'Advocacy Organizations': 'Law and Government',
    'Pornography': 'Adult',
    'Other Adult Materials': 'Adult',
    'Nudity and Risque': 'Adult',
    'Sex Education': 'Adult',
    'Reference': 'Reference',
    'Online Meeting': 'Online Technologies',
    'Web Chat': 'Online Technologies',
    'Remote Access': 'Online Technologies',
    'Web-based Email': 'Online Technologies',
    'Instant Messaging': 'Online Technologies',
    'Web-based Applications': 'Online Technologies',
    'Malicious Websites': 'Malicious webpages',
    'Phishing': 'Malicious webpages',
    'Spam URLs': 'Malicious webpages',
    'Potentially Unwanted Program': 'Malicious webpages',
    'Illegal or Unethical': 'Malicious webpages',
    'Shopping': 'Shopping',
    'Advertising': 'Advertising',
    'Information and Computer Security': 'Info Security',
    'Secure Websites': 'Info Security',
    'Real Estate': 'Real Estate',
    'Global Religion': 'Religion',
    'Alternative Beliefs': 'Religion',
    'Tobacco': 'Alcohol and Tobacco',
    'Alcohol': 'Alcohol and Tobacco',
    'Content Servers': 'Hosting and Sharing',
    'Web Hosting': 'Hosting and Sharing',
    'File Sharing and Storage': 'Hosting and Sharing',
    'Streaming Media and Download': 'Hosting and Sharing',
    'Domain Parking': 'Hosting and Sharing',
    'Peer-to-peer File Sharing': 'Hosting and Sharing',
    'General Organizations': 'Organizations',
    'Charitable Organizations': 'Organizations',
     'Meaningless Content': 'Meaningless Content',
}

In [12]:
# So we have final categories:

final_categories = ['Internet and Telecom',
'Arts and Entertainment',
'Business and Industry',
'Travel',
'Health',
'Games',
'People and Society',
'Finance',
'Sports',
'News and Media',
'Career and Education',
'Gambling',
'Food and Drink',
'Autos and Vehicles',
'Law and Government',
'Adult',
'Reference',
'Online Technologies',
'Malicious webpages',
'Shopping',
'Advertising',
'Info Security',
'Real Estate',
'Religion',
'Alcohol and Tobacco',
'Hosting and Sharing', 
'Organizations',
'Meaningless Content']

In [13]:
# And these we will not use at all:

categories_unused = [
    'Not Rated',
    'Newly Observed Domain',
    'Proxy Avoidance',
    'Web Analytics',
    'Job Search',
    'Hacking',
    'Dynamic DNS',
    'Dating',
    'Armed Forces',
    'Weapons (Sales)',
    'Crypto Mining',
    'Newly Registered Domain',
    'Drug Abuse',
    'Abortion',
    'URL Shortening',
    'Plagiarism',
    'Folklore',
    'Explicit Violence',
    'Lingerie and Swimsuit',
    'Discrimination',
    'Digital Postcards',
    'Marijuana',
    'Dynamic Content',
    'Child Sexual Abuse',
    'Child Abuse',
    'Extremist Groups',
    'Homosexuality',
    'Terrorism',
    'Personal Privacy'
]

In [27]:
# Here we can make groups for categories (93 categories -> 28 categories):
df['category'] = df['category'].replace(category_mapping)

# And delete some of them:
df = df.loc[~df['category'].isin(categories_unused)]
df = df.reset_index(drop=True)

In [28]:
# The same we do for train:

train_df['category'] = train_df['category'].replace(category_mapping)
train_df = train_df.loc[~train_df['category'].isin(categories_unused)]
train_df = train_df.reset_index(drop=True)

# And test:

test_df['category'] = test_df['category'].replace(category_mapping)
test_df = test_df.loc[~test_df['category'].isin(categories_unused)]
test_df = test_df.reset_index(drop=True)

In [35]:
# And finally save it:

df.to_csv(path + 'ten_million_url_list.csv', index=False)
train_df.to_csv(path + 'train.csv', index=False)
test_df.to_csv(path + 'test.csv', index=False)